In [24]:
import requests as rq
from bs4 import BeautifulSoup

url = 'https://finance.naver.com/sise/sise_deposit.naver'
data = rq.get(url)
data_html = BeautifulSoup(data.content)

parse_day = data_html.select_one('div.subtop_sise_graph2 > ul.subtop_chart_note > li > span.tah').text

import re

biz_day = re.findall('[0-9]+', parse_day)
biz_day = ''.join(biz_day)

from io import BytesIO
import pandas as pd

gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_stk = {
    'locale': 'ko_KR',
    'mktId': 'STK',
    'trdDd': biz_day,
    'money': '1',
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03901'
}
headers = {'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020506'}
otp_stk = rq.post(gen_otp_url, gen_otp_stk, headers=headers).text

down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
down_sector_stk = rq.post(down_url, {'code': otp_stk}, headers=headers)

sector_stk = pd.read_csv(BytesIO(down_sector_stk.content), encoding='EUC-KR')

gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_ksq = {
    'locale': 'ko_KR',
    'mktId': 'KSQ',
    'trdDd': biz_day,
    'money': '1',
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03901'
}
headers = {'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020506'}
otp_ksq = rq.post(gen_otp_url, gen_otp_ksq, headers=headers).text
down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
down_sector_ksq = rq.post(down_url, {'code': otp_ksq}, headers=headers)
sector_ksq = pd.read_csv(BytesIO(down_sector_ksq.content), encoding='EUC-KR')
krx_sector = pd.concat([sector_stk, sector_ksq]).reset_index(drop = True)
krx_sector['종목명'] = krx_sector['종목명'].str.strip()
krx_sector['기준일'] = biz_day

In [25]:
krx_sector

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액,기준일
0,095570,AJ네트웍스,KOSPI,서비스업,4475,15,0.34,209529770125,20230517
1,006840,AK홀딩스,KOSPI,기타금융,20650,350,1.72,273562134650,20230517
2,027410,BGF,KOSPI,기타금융,4095,-10,-0.24,391960259145,20230517
3,282330,BGF리테일,KOSPI,유통업,180100,-2100,-1.15,3112831470600,20230517
4,138930,BNK금융지주,KOSPI,기타금융,6670,20,0.30,2173988090820,20230517
...,...,...,...,...,...,...,...,...,...
2580,024060,흥구석유,KOSDAQ,유통,5540,-10,-0.18,83100000000,20230517
2581,010240,흥국,KOSDAQ,기계·장비,6620,210,3.28,81576247520,20230517
2582,189980,흥국에프엔비,KOSDAQ,음식료·담배,2840,45,1.61,113991428680,20230517
2583,037440,희림,KOSDAQ,기타서비스,9120,0,0.00,126972972000,20230517
